## 1. IMDB

영화 정보 제공 웹사이트 -> 리뷰들을 감정분석용 데이터 셋으로 많이 씀
kaggle 의 감성분석 튜토리얼 진행

*  영어 데이터 사용
*  train set & test set 모두 25,000 개의 긍정 부정 리뷰로 구성 (긍정-label 1 , 부정 - label 0)


## 2. Load Files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
import matplotlib as plt

In [ ]:
# tsv 파일은 탭으로 구분된 파일
# header=0 은 첫 행이 열 이름
# delimiter="\t" 파일이 탭으로 구분됨
# quoting = 3, 따옴표 인식 x, 일반 문자 처리

train = pd.read_csv('/content/drive/MyDrive/24-winter KUBIG NLP/WEEK 2/WEEK 2 복습과제 IMDB 텍스트 감성분석/labeledTrainData.tsv', header=0, delimiter="\t", quoting=3)
test = pd.read_csv('/content/drive/MyDrive/24-winter KUBIG NLP/WEEK 2/WEEK 2 복습과제 IMDB 텍스트 감성분석/testData.tsv', header=0, delimiter='\t', quoting=3)
submit = pd.read_csv('/content/drive/MyDrive/24-winter KUBIG NLP/WEEK 2/WEEK 2 복습과제 IMDB 텍스트 감성분석/sampleSubmission.csv')

## 3. 데이터 탐색


In [ ]:
train.info()

In [ ]:
train.head()

In [ ]:
test.info()

In [ ]:
test.head()

In [ ]:
train['sentiment'].value_counts()

## 4. Text Preprocessing

### 4-1. html 태그 제거

리뷰에서는 `<br>` 이라는 html 줄바꿈 태그가 보임!

`BeautifulSoup` 를 이용해 태그 지우자

In [ ]:
train["review"][0]

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
example1 = BeautifulSoup(train["review"][0])
example1 = example1.get_text()
print("before deleting:" , train["review"][0])
print("after deleting:" , example1)

### 4-2. 정규표현식(re)로 알파벳만 남기기

html 태그는 제거했으나, 각종 특수문자가 여전히 남아있음
re를 통해 알파벳이 아닌것을 공백으로 대체

In [ ]:
import re
letters_only = re.sub("[^a-zA-Z]", " ", example1)

In [ ]:
train["review"][0]

In [ ]:
letters_only

### 4-3. 토큰화 (Tokenizing)

*토큰은 의미를 갖는 최소분석단위 (한국어는 형태소 , 영어는 띄어쓰기 단위)
*토큰화는 corpus 덩어리를 작은 토큰 단위로 쪼개주는 자겅ㅂ

split() 함수를 써주면, 띄어쓰기로 토큰화
lower_case 함수로 소문자 변환하는 이유는 대소문자가 다른 단어로 구분되기 때문에 복잡성 저하

In [ ]:
lower_case = letters_only.lower()
token_words = lower_case.split()
print("토큰화 이후 생성된 토큰(단어) 개수", len(token_words))

아래는 nltk 와 keras 에서 토큰화를 수행해주는 도구들

In [ ]:
# from nltk.tokenize import word_tokenize
# word_tokenize(lower_case)
# from nltk.tokenize import WordPunctTokenizer
# WordPunctTokenizer().tokenize(lower_case)
# from tensorflow.keras.preprocessing.text import text_to_word_sequence
# text_to_word_sequence(lower_case)

### 4-4.불용어 제거

i we our 같이 크게 중요하지 않은 단어들 제거

nltk 내장 불용어는 179개

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
stopwords_list = stopwords.words('english')
print("nltk에 내장된 불용어 개수: ", len(stopwords_list))
print("불용어 예시: ", stopwords_list[:10])
non_stopwords = [w for w in token_words if not w in stopwords_list]
print("예시 review에서 불용어 제거하고 남은 토큰 개수: ",len(non_stopwords))

### 4-5. 어간 추출

nltk 에선 어간을 추출해주는 도구를 제공합니다.
100% 정확하지는 않음

* ALIZE → AL
* ANCE → 제거
* ICAL → IC

* formalize → formal
* allowance → allow
* electricical → electric

In [ ]:
from nltk.stem import PorterStemmer
porter_stemmer = PorterStemmer()
stemmed_words = [porter_stemmer.stem(w) for w in non_stopwords]

In [ ]:
stemmed_words[1:10]

### 4-6. 표제어 추출

표제어 추출은 어간 추출과 다르게 어떤 품사로 쓰였는지를 고려함

stemming(어간 추출)을 하는 것보다 lemmatization(표제어 추출)을 하는 것이 더 효과적

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [ ]:
lemmatized_words = [wordnet_lemmatizer.lemmatize(w) for w in non_stopwords]
lemmatized_words = [wordnet_lemmatizer.lemmatize(w,"v") for w in lemmatized_words]

In [ ]:
print(non_stopwords[21:40]) # none
print(stemmed_words[21:40]) # stemming
print(lemmatized_words[21:40]) # lemmatization

### 하나의 함수로 표현

앞선 6가지 전처리 과정을 하나의 함수로 통합

In [ ]:
stopwords_list= set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()

def review_to_words(raw_review):
    except_tag = BeautifulSoup(raw_review).get_text() # html 태그 제거
    letters_only = re.sub("[^a-zA-Z]", " ", except_tag) # 정규표현식으로 알파벳 남기기
    token_words = letters_only.lower().split() # 소문자로 통합 후 토큰화
    non_stopwords = [w for w in token_words if not w in stopwords_list] # 불용어 제거
    lemmatized_words = [wordnet_lemmatizer.lemmatize(w) for w in non_stopwords] # 표제어 추출
    lemmatized_words = [wordnet_lemmatizer.lemmatize(w,"v") for w in lemmatized_words]
    words = " ".join(lemmatized_words)
    return words

## 5. BoW (Bag of Words) 형태로 변환

단어의 출현 빈도에 집중해 텍스트를 수치화하는 표현 방식

예를 들어 아래처럼 두개의 노래구절이 있는 경우

In [ ]:
lyric1 = "but I don't want to stay in the middle" # ditto
lyric2 = "like you a little don't want no riddle"

In [ ]:
vocab = ["but", "I", "don't", "want", "to", "stay", "in", "the", "middle", "like", "you", "a", "little", "no", "riddle"]

In [ ]:
# 각 단어에 인덱스 부여
# 인덱스 위치에 단어의 빈도 표현

lyric1 = [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
lyric2 = [0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]

In [ ]:
num_reviews = train['review'].size

clean_train_reviews = []
for i in range(0, num_reviews):
     if (i + 1) % 5000 == 0 :  #실행이 잘되는지 확인하기 위해 5000개 실행될때마다 확인문구
         print('Review {} of {}'.format(i+1, num_reviews))
     clean_train_reviews.append(review_to_words(train['review'][i]))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = 'word', # 학습 단위
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = None,
                             min_df = 2, # 토큰이 나타날 최소 문서 개수
                             ngram_range=(1, 2), # 단어의 묶음 개수
                             max_features = 4000) # 토큰의 최대 개수, 즉 컬럼의 최대 개수

## 참고 TF-IDF

단어 빈도, 문서빈도 역수 사용

* 단어빈도 : 특정 단어가 한 문서 내에서 출현한 빈도
* 문서빈도 : 특정 단어가 출현한 전체 문서 개수

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df = 4, # 토큰이 나타날 최소 문서 개수
                             analyzer = 'word', # 학습 단위
                             ngram_range = (1, 2), # 단어의 묶음 개수
                             max_features = 1000) # 토큰의 최대 개수, 즉 컬럼의 최대 개수

In [ ]:
train_data_features = vectorizer.fit_transform(clean_train_reviews)
train_data_features.shape

In [ ]:
vocab = vectorizer.get_feature_names()
vocab[:10]

## 7. Modeling

baseline 코드로 제시된 트리 계열 알고리즘으로 학습 시키기

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

x = train_data_features
y = train['sentiment']

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3, random_state=42)

# 트리 알고리즘 3개를 사용
rf = RandomForestClassifier(n_estimators = 200,
                            n_jobs = -1,
                            random_state=42,
                            max_depth=20)

xgb = XGBClassifier(n_estimators=200,
                    max_depth=10,
                    learning_rate=0.05,
                    objective='binary:logistic')

lgbm = LGBMClassifier(n_estimators=200,
                    max_depth=10,
                    metric='binary_logloss')

In [ ]:
xgb.fit(x_train, y_train)
y_pred_xgb = xgb.predict_proba(x_val)

In [ ]:
lgbm.fit(x_train, y_train)
y_pred_lgbm = lgbm.predict_proba(x_val)

In [ ]:
rf.fit(x_train, y_train)
y_pred_rf = rf.predict_proba(x_val)

학습된 모델들의 roc_auc_score 구하기

In [ ]:
print('Random Forest AUC Score :', roc_auc_score(y_val, y_pred_rf[:,1]))
print('XGBoost AUC Score :', roc_auc_score(y_val, y_pred_xgb[:,1]))
print('LGBM AUC Score :', roc_auc_score(y_val, y_pred_lgbm[:,1]))

## 8. test set 추론  

In [ ]:
num_reviews = test['review'].size

clean_test_reviews = []
for i in range(0, num_reviews):
     if (i + 1) % 5000 == 0 :  #실행이 잘 되는지 확인하기 위해 5000개 실행될때마다 확인문구
         print('Review {} of {}'.format(i+1, num_reviews))
     clean_test_reviews.append(review_to_words(test['review'][i]))

In [ ]:
test_data_features = vectorizer.fit_transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

In [ ]:
# 3개 알고리즘 중 원하는 것으로 predict
result = lgbm.predict(test_data_features)
submit['sentiment'] = result

In [ ]:
submit.head(10)